**Module 3: DataFrames and Datasets**

### Understanding DataFrames and Datasets

**1. What is a Schema?**
A schema defines the structure of a dataset, specifying the column names, data types, and constraints. It ensures that the data adheres to a predefined format, enabling consistency and compatibility across processing workflows.

- **Benefits of Schema:**
  - Enforces data integrity and validation.
  - Enables query optimization.
  - Facilitates compatibility with downstream applications.

*Example*: Viewing the schema of a DataFrame:

**2. DataFrames**
A DataFrame is a distributed collection of data organized into named columns, similar to a table in a relational database. It is designed to support both structured and semi-structured data processing.

- **Key Features of DataFrames:**
  - Schema-based processing.
  - Lazy evaluation for efficient execution.
  - Catalyst optimizer for query planning.
  - Multi-language support: Python, Scala, SQL, R.

*Example*: Loading a JSON dataset:

In [0]:
# Load JSON data
flight_data_json = "dbfs:/mnt/data/data/flight-data/json/2015-summary.json"
df = spark.read.format("json").load(flight_data_json)

df.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|  ABC|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [0]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: string (nullable = true)



**3. Datasets**
Datasets provide type-safe operations with compile-time type checking, combining the benefits of RDDs with the optimizations of DataFrames. While primarily used in Scala and Java, the equivalent in Python is the DataFrame API.

- **Differences Between DataFrames and Datasets:**
  - **Type Safety**: Datasets offer type safety, while DataFrames do not.
  - **Languages**: Datasets are native to Scala and Java; DataFrames are universal.
  - **Serialization**: Datasets use encoders for object serialization.


In [0]:
%scala
import org.apache.spark.sql.{SparkSession, Dataset}
import org.apache.spark.sql.Encoders

import org.apache.spark.sql.{SparkSession, Dataset}
import org.apache.spark.sql.Encoders

In [0]:
%scala
// Define a case class that represents the schema
case class FlightData(DEST_COUNTRY_NAME: String, ORIGIN_COUNTRY_NAME: String, count: Long)

// Load data into a Dataset
val flightDataDS: Dataset[FlightData] = spark.read
  .option("header", "true")
  .schema("DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG")
  .csv("dbfs:/mnt/data/data/flight-data/csv/2015-summary.csv")
  .as[FlightData] // Converts DataFrame to Dataset with type safety

defined class FlightData
flightDataDS: org.apache.spark.sql.Dataset[FlightData] = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [0]:
%scala
// Show first few records
flightDataDS.show(5)

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| Romania| 15|
 United States| Croatia| 1|
 United States| Ireland| 344|
 Egypt| United States| 15|
 United States| India| 62|
+-----------------+-------------------+-----+
only showing top 5 rows

In [0]:
%scala
// Filter using strongly typed API
val filteredDS = flightDataDS.filter(_.count > 1000)
filteredDS.show(5)


+-----------------+-------------------+------+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
 Mexico| United States| 7140|
 United States| Dominican Republic| 1420|
 United States| United States|370002|
 Germany| United States| 1468|
 Canada| United States| 8399|
+-----------------+-------------------+------+
only showing top 5 rows

filteredDS: org.apache.spark.sql.Dataset[FlightData] = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [0]:
%scala
import org.apache.spark.sql.functions._

val aggregatedDS = flightDataDS
  .groupByKey(_.DEST_COUNTRY_NAME) // Grouping by key
  .agg(sum("count").as[Long]) // Type-safe aggregation

aggregatedDS.show(10)


+-----------+----------+
 key|sum(count)|
+-----------+----------+
 Anguilla| 41|
 Russia| 176|
 Paraguay| 60|
 Senegal| 40|
 Sweden| 118|
 Kiribati| 26|
 Guyana| 64|
Philippines| 134|
 Djibouti| 1|
 Malaysia| 2|
+-----------+----------+
only showing top 10 rows

import org.apache.spark.sql.functions._
aggregatedDS: org.apache.spark.sql.Dataset[(String, Long)] = [key: string, sum(count): bigint]

In [0]:
%scala
// Transforming dataset using the map function
val transformedDS = flightDataDS.map(flight =>
  flight.copy(count = flight.count * 2) // Doubling count
)

transformedDS.show(5)


+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| Romania| 30|
 United States| Croatia| 2|
 United States| Ireland| 688|
 Egypt| United States| 30|
 United States| India| 124|
+-----------------+-------------------+-----+
only showing top 5 rows

transformedDS: org.apache.spark.sql.Dataset[FlightData] = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [0]:
%scala
import org.apache.spark.sql.DataFrame

// Convert Dataset to DataFrame
val flightDataDF: DataFrame = flightDataDS.toDF()

// Convert DataFrame to Dataset
val flightDataDS2: Dataset[FlightData] = flightDataDF.as[FlightData]


import org.apache.spark.sql.DataFrame
flightDataDF: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]
flightDataDS2: org.apache.spark.sql.Dataset[FlightData] = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

### File Formats in Spark

**1. JSON (JavaScript Object Notation)**
A lightweight, semi-structured format ideal for data exchange.

- **Advantages:**
  - Human-readable.
  - Flexible structure for nested data.

*Example*: Reading and writing JSON:

In [0]:
# Reading JSON
json_df = spark.read.format("json").load("dbfs:/mnt/data/data/flight-data/json/2015-summary.json")

# Writing JSON
json_df.write.format("json").save("dbfs:/mnt/data/output/flight-data-json")

**2. Parquet**
A columnar storage format optimized for analytics, offering efficient compression and query performance.

- **Advantages:**
  - Supports schema evolution.
  - Optimized for columnar operations.

*Example*: Working with Parquet:

In [0]:
# Reading Parquet
parquet_df = spark.read.format("parquet").load("dbfs:/mnt/data/data/flight-data/parquet/2010-summary.parquet")

# Writing Parquet
parquet_df.write.format("parquet").save("dbfs:/mnt/data/output/flight-data-parquet")

**3. Avro**
A row-based binary format commonly used for data serialization.

- **Advantages:**
  - Compact and efficient.
  - Supports schema definition and evolution.

*Example*: Working with Avro:

In [0]:
parquet_df.write.format("avro").save("dbfs:/mnt/data/output/flight-data-avro")



In [0]:
# Reading Avro
avro_df = spark.read.format("avro").load("dbfs:/mnt/data/output/flight-data-avro")
avro_df.display()

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,1
United States,Ireland,264
United States,India,69
Egypt,United States,24
Equatorial Guinea,United States,1
United States,Singapore,25
United States,Grenada,54
Costa Rica,United States,477
Senegal,United States,29
United States,Marshall Islands,44


**4. CSV (Comma-Separated Values)**
A widely used format for simple tabular data.

- **Advantages:**
  - Easy to understand and use.
  - Compatible with many tools.

*Example*: Working with CSV:

In [0]:
# Reading CSV
csv_df = spark.read.format("csv").option("header", True).load("dbfs:/mnt/data/data/flight-data/csv/2010-summary.csv")

# Writing CSV
csv_df.write.format("csv").option("header", True).save("dbfs:/mnt/data/output/flight-data-csv")

---

### Schema Management and Optimization

**1. Inferring Schema**
Spark automatically detects column names and data types based on file contents.

*Example*: Inferring schema from JSON:

In [0]:
schema_inferred_df = spark.read.json(flight_data_json)
schema_inferred_df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: string (nullable = true)



**2. Explicit Schema Definition**
Define schemas for strict data validation and compatibility.

*Example*: Defining a schema:

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, LongType

schema = StructType([
    StructField("DEST_COUNTRY_NAME", StringType(), True),
    StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
    StructField("count", LongType(), True)
])

schema_defined_df = spark.read.format("json").schema(schema).load(flight_data_json)
schema_defined_df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



---



---